In [1]:
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf

## Prepare data

In [3]:
har_x_train, har_y_train, list_ch_train = read_data(data_path="./dataset/UCI HAR Dataset/", split="train") # train
har_x_test, har_y_test, list_ch_test = read_data(data_path="./dataset/UCI HAR Dataset/", split="train") # test

In [4]:
har_x_train, har_x_test = standardize(har_x_train, har_x_test)

In [5]:
har_y_train = har_y_train - 1

In [30]:
har_y_test = har_y_test - 1

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(har_x_train, har_y_train, stratify=har_y_train, random_state=42)

In [7]:
y_train_onehot = one_hot(y_train)
y_train_valid = one_hot(y_valid)
y_test_onehot = one_hot(har_y_test)

In [8]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

# Build CNN model

## Hyperparameters

In [21]:
batch_size = 60
seq_len = 128
learning_rate = 0.01
n_epochs = 100

n_classes = 6
n_channels = 9

## the model

In [22]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, seq_len, n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

In [23]:
with tf.name_scope("cnn"):
    
    # Convolutional layers
    conv1 = tf.layers.conv1d(X, 
                             filters=18, 
                             kernel_size=2, 
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)
    max_pool1 = tf.layers.max_pooling1d(conv1,
                                 pool_size=4,
                                 strides=4,
                                 padding="same")
    conv2 = tf.layers.conv1d(max_pool1,
                             filters=36, 
                             kernel_size=2,
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)
    max_pool2 = tf.layers.max_pooling1d(conv2,
                                        pool_size=4,
                                        strides=4,
                                        padding="same")
    conv3 = tf.layers.conv1d(max_pool2,
                             filters=72,
                             kernel_size=2,
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)
    max_pool3 = tf.layers.max_pooling1d(conv3,
                                        pool_size=4,
                                        strides=4,
                                        padding="same")
    
    # Fully Connected layers
    flat = tf.reshape(max_pool3, (-1, 2*72))
    drop = tf.nn.dropout(flat, keep_prob=0.8)
    
    logits = tf.layers.dense(drop, n_classes)
    
    

In [24]:
logits.get_shape()

TensorShape([Dimension(None), Dimension(6)])

In [25]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [26]:
with tf.name_scope("train"):
    training_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [27]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [28]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [36]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for X_batch, y_batch in get_batches(X_train, y_train, batch_size):
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch})
        valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        print("Epoch: ", epoch, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./har_cnn_model.ckpt")

Epoch:  0 Valid accuracy:  0.775299
Epoch:  1 Valid accuracy:  0.832971
Epoch:  2 Valid accuracy:  0.849293
Epoch:  3 Valid accuracy:  0.869423
Epoch:  4 Valid accuracy:  0.821545
Epoch:  5 Valid accuracy:  0.855822
Epoch:  6 Valid accuracy:  0.848749
Epoch:  7 Valid accuracy:  0.855277
Epoch:  8 Valid accuracy:  0.872144
Epoch:  9 Valid accuracy:  0.85691
Epoch:  10 Valid accuracy:  0.825898
Epoch:  11 Valid accuracy:  0.883025
Epoch:  12 Valid accuracy:  0.902067
Epoch:  13 Valid accuracy:  0.865615
Epoch:  14 Valid accuracy:  0.89445
Epoch:  15 Valid accuracy:  0.890098
Epoch:  16 Valid accuracy:  0.884657
Epoch:  17 Valid accuracy:  0.88901
Epoch:  18 Valid accuracy:  0.898259
Epoch:  19 Valid accuracy:  0.890098
Epoch:  20 Valid accuracy:  0.865071
Epoch:  21 Valid accuracy:  0.888466
Epoch:  22 Valid accuracy:  0.897715
Epoch:  23 Valid accuracy:  0.880849
Epoch:  24 Valid accuracy:  0.898259
Epoch:  25 Valid accuracy:  0.881937
Epoch:  26 Valid accuracy:  0.887378
Epoch:  27 Val

In [37]:
with tf.Session() as sess:
    saver.restore(sess, "./har_cnn_model.ckpt")
    test_acc = accuracy.eval(feed_dict={X: har_x_test, y: har_y_test})
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./har_cnn_model.ckpt
Test Accuracy:  0.949401
